In [1]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import BertConfig, BertModel

2025-05-29 11:42:56.398291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748511776.420060    1614 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748511776.425682    1614 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748511776.441830    1614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748511776.441851    1614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748511776.441853    1614 computation_placer.cc:177] computation placer alr

------------

### Tokenizer

In [7]:
# Fetching model’s tokenizer and caching it
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


The output is a dictionary containing:
- input_ids: unique identifiers of the tokens in each sentence
- attention_mask:
  
It can be saved with ```tokenizer.save_pretrained("directory_on_my_computer")```

In [16]:
# How words are tokenized and how ids can be matched back to vocabulary
tokens = tokenizer.tokenize(sequences)
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
decoded_string = tokenizer.decode(ids)
print(decoded_string)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', 'i', 'hate', 'this', 'so', 'much', '!']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 1045, 5223, 2023, 2061, 2172, 999]
i've been waiting for a huggingface course my whole life. i hate this so much!


-----------

### Automodels:

In [6]:
# Same way to download the pretrained model
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

The AutoModel class and all of its relatives are actually simple wrappers over the wide variety of models available in the library. It’s a clever wrapper as it can automatically guess the appropriate model architecture for your checkpoint.

In [7]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


- Batch size: The number of sequences processed at a time (2 in our example).
- Sequence length: The length of the numerical representation of the sequence (16 in our example).
- Hidden size: The vector dimension of each model input.

In [11]:
# Model with a sequence classification head
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 2])


In [ ]:
# Printing number of parameters in the model
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel() # Sums the count of all elements in a certain parameter tensor
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

In [13]:
# Logits, unnormalized scores, are converted to probabilities through a SoftMax layer
print(outputs.logits)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [14]:
# Get the labels corresponding to each position,
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

-----------

### Non-automodels

In [3]:
# Loading configuration to the model
config = BertConfig()
model = BertModel(config)
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



The model needs to be trained first (a lot of time), but a loaded version can be fetched.

In [4]:
# Loading a Transformer model that is already trained
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

More Bert checkpoints in https://huggingface.co/models?other=bert. Unlike automodels (architecture-agnostic), only Bert checkpoinys will work with Bert.

Saving a model with ```model.save_pretrained("directory_on_my_computer")``` creates two files:
- config.json: architecture attributes and metadata
- pytorch_model.bin: state dictionary with weights

--------

### Handling multiple sequences - Part 1

In [17]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids]) # Transformers models expect multiple sentences by default
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Batching is the act of sending multiple sentences through the model, all at once. Batches must have a rectangular shape, so padding us used

In [18]:
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


There’s something wrong with the logits in our batched predictions in the second row: different values were obtained because the key feature of Transformer models is attention layers that contextualize each token. These will take into account the padding tokens since they attend to all of the tokens of a sequence. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.

In [19]:
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]
outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [ ]:
# Different paddings
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
model_inputs = tokenizer(sequences, padding="longest") # Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="max_length") # Will pad the sequences up to the model max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8) # Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, truncation=True) # Will truncate the sequences that are longer than the model max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True) # Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt") # Returns PyTorch (pt), TensorFlow (tf) or NumPy (np) tensors

model_inputs = tokenizer(sequences, padding=True, truncation=True, )

In [15]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model_inputs = tokenizer("I've been waiting for a HuggingFace course my whole life.",  # Sequences are handled as a pair so the model can predict if two sentences are paraphrases or not
                         "I hate this so much!",
                         padding=True, # Usually the two parameters are used:
                         truncation=True)
print(model_inputs)
print(tokenizer.convert_ids_to_tokens(model_inputs["input_ids"]))

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 1045, 5223, 2023, 2061, 2172, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', '[SEP]', 'i', 'hate', 'this', 'so', 'much', '!', '[SEP]']


The model added special words for pretraining to separate the phrases. Notice that ```token_type_ids``` has different IDs for each phrase. This key may not be on a different checkpoint. BERT in this case needs it for ```next sentence prediction```. However, ```token_type_ids``` is not compulsory as long as you use the same checkpoint for the tokenizer and the model.